In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [54]:
import pandas
import gensim

import matplotlib.pyplot as plt
import matplotlib

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Ridge

matplotlib.style.use('ggplot')

## 1. Read data

In [3]:
data_file = "../data/bdata.20130222.mhci.public.1.txt"

data = pandas.read_csv(data_file, sep = '\t')

In [4]:
data.head()

,species,mhc,peptide_length,cv,sequence,inequality,meas
0,cow,BoLA-HD6,9,TBD,ALFYKDGKL,=,1.000000
1,cow,BoLA-HD6,9,TBD,ALYEKKLAL,=,1.000000
2,cow,BoLA-HD6,9,TBD,AMKDRFQPL,=,4.521706
3,cow,BoLA-HD6,9,TBD,AQRELFFTL,=,1.000000
4,cow,BoLA-HD6,9,TBD,FMKVKFEAL,=,1.576747


In [5]:
print "data shape:", data.shape

data shape: (179692, 7)


In [6]:
print "list of species:", unique(data['species'])

list of species: ['None' 'chimpanzee' 'cow' 'gorilla' 'human' 'macaque' 'mouse']


In [7]:
print "number of unique mhc:", len(unique(data['mhc']))

number of unique mhc: 161


In [8]:
print "number of unique sequences:", len(unique(data['sequence']))

number of unique sequences: 42507


In [9]:
data['meas'].describe()

count    1.796920e+05
mean     2.112472e+04
std      5.986279e+04
min      1.000000e+00
25%      3.589934e+02
50%      1.474869e+04
75%      2.000000e+04
max      1.427660e+07
Name: meas, dtype: float64

 ## 2. Sequences to vector space

Is it good idea? Who knows... Paragraph2Vec stuff

In [10]:
s_file = '../data/seqs.txt'

sentences = gensim.models.doc2vec.TaggedLineDocument(s_file)

model = gensim.models.Doc2Vec(sentences, size = 32, window = 4, workers = 4)

In [74]:
# select one mhc
vecs_df = pandas.DataFrame.from_records(model.docvecs)

selected_mhc = 'HLA-A*03:01'

indexes = data['mhc'][data['mhc'] == selected_mhc].index
selected_X = vecs_df.iloc[indexes]
selected_y = data['meas'][indexes]

selected_y.describe()

count    7.089000e+03
mean     1.960290e+04
std      4.099206e+04
min      1.000000e+00
25%      2.811908e+02
50%      1.267504e+04
75%      2.000000e+04
max      1.831270e+06
Name: meas, dtype: float64

## 3. Learn something

In [75]:
#normalize output
m = selected_y.mean()
d = selected_y.max() - selected_y.min()
selected_y = (selected_y - m) / d

# train/test split
random_number = 42
X_train, X_test, y_train, y_test = train_test_split(selected_X, selected_y,
                                                    test_size = 0.33, random_state = random_number)

In [86]:
def ridge_regression(X, y, alpha):
    #Fit the model
    ridgereg = Ridge(alpha=alpha,normalize=True)
    ridgereg.fit(X, y)
    y_pred = ridgereg.predict(X)
    
    #Return the result in pre-defined format
    rss = sum((y_pred - y)**2)
    return rss, ridgereg.intercept_, ridgereg.coef_, ridgereg

In [88]:
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

col = ['coef_x_%d'%i for i in range(1,33)]
ind = ['alpha_%.2g'%alpha_ridge[i] for i in range(0,10)]
coef_matrix_ridge = pandas.DataFrame(index = ind, columns = col)
rss = [0] * 10
intercept = [0] * 10
predictors = [0] * 10

for i in range(10):
    rss[i], intercept[i], coef_matrix_ridge.iloc[i,], predictors[i] = ridge_regression(X_train, y_train, alpha_ridge[i])

In [107]:
# predict test data

test_rss = [0] * 10
for i in range(10):
    m_pred = predictors[i]
    test_pred = m_pred.predict(X_test)
    test_rss[i] = sum((test_pred - y_test)**2)
    
    print '-' * 10
    print "alpha = ", alpha_ridge[i]
    print "train rss = ", rss[i]
    print "test rss = ", test_rss[i]
    

----------
alpha =  1e-15
train rss =  2.96778133064
test rss =  0.535133372552
----------
alpha =  1e-10
train rss =  2.96778133064
test rss =  0.535133372549
----------
alpha =  1e-08
train rss =  2.96778133064
test rss =  0.535133372259
----------
alpha =  0.0001
train rss =  2.96778133111
test rss =  0.535130442098
----------
alpha =  0.001
train rss =  2.96778137784
test rss =  0.535104116582
----------
alpha =  0.01
train rss =  2.96778596383
test rss =  0.534845597849
----------
alpha =  1
train rss =  2.97961832777
test rss =  0.526536482644
----------
alpha =  5
train rss =  3.00238579004
test rss =  0.528155226714
----------
alpha =  10
train rss =  3.00972923034
test rss =  0.529566937995
----------
alpha =  20
train rss =  3.01436703318
test rss =  0.530584773606
